In [ ]:
# Global data variables
SANDBOX_NAME = 'fesp'# Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/data/" 

In [ ]:
from pyspark.sql import functions as F



## Ejercicio 1

A partir del siguiente Dataframe:


In [ ]:
file_name = "TA_restaurants_curated.csv"

df = spark.read.csv(DATA_PATH + file_name, sep=',', header=True, inferSchema=True)
df.show()



1) Determina el porcentaje de valores nulos de cada columna.

2) Renombre las columnas de forma que todas cumplan las buenas prácticas.

3) Borre la columna URL_TA

4) Reemplace los valores nulos de las columnas numéricas por su valor medio.

5) Calcule el número de restaurantes recogidos para cada ciudad.

6) Calcule el rating medio por ciudad.

7) Genere un nuevo DataFrame con aquellos restaurantes italianos que tienen una valoración igual a 5.

8) Determine cuál es la ciudad con el máximo número de reviews.





In [ ]:
# Respuesta
rows = df.count()

for col in df.columns:
    
    n_missing = df.filter(F.col(col).isNull()).count()
    perc_missing = 100 * n_missing / rows
    
    print(col, round(perc_missing, 2))

In [ ]:
# Respuesta
def rename_function(c):
    return c.strip().lower().replace('.', '').replace(' ', '_').replace("_c0", "index")
for c in df.columns:
    df = df.withColumnRenamed(c, rename_function(c))

In [ ]:
# Respuesta
df = df.drop('url_ta')

In [ ]:
# Respuesta
df.printSchema()

In [ ]:
# Respuesta
avgs = df.agg({"ranking": "avg", "rating": "avg", "number_of_reviews":"avg"}).first()
avgs

In [ ]:
# Respuesta
df.fillna(0, subset=['ranking','rating','number_of_reviews']).show()

In [ ]:
# Respuesta
rest_per_city = df.groupBy(F.col('city')).count()
rest_per_city.show()

In [ ]:
# Respuesta
rating_per_city = df.groupBy('city').agg(F.round(F.avg(F.col('rating')),2))
rating_per_city.show()

In [ ]:
# Respuesta
df.where((F.col('cuisine_style').like('%Italian%')) & (F.col('rating') == 5)).show()

In [ ]:
# Respuesta
reviews_per_city = df.groupBy('city').agg(F.sum(F.col('number_of_reviews')).alias('sum_reviews')).sort(F.col('sum_reviews').desc())
reviews_per_city.show()

In [ ]:
# Respuesta
maximo = reviews_per_city.select(F.max("sum_reviews")).first()[0]
reviews_per_city.filter(F.col("sum_reviews") == maximo).select("city").show()